<a href="https://colab.research.google.com/github/chuan137/egohands/blob/master/grocery/grocery_categories_v06_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Note

__Description__ more categories for training.

__Update__


## Training Details

**Train data**
* 5 categories: BEANS, CAKE, CANDY, MILK, PASTA 
* image size 224

**Architecture**
* **With** fully connected layer
* prediction layer with L2 regularizer

**Hyper parameters**
* softmax layer with regularizer, weight = 0.0001
* optimizer Adam, learning rate 0.0001



## Prepare

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cp /content/drive/My\ Drive/freiburg_groceries_dataset.zip .
!unzip -q freiburg_groceries_dataset.zip

## Load Data

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


* split train and test images

In [0]:
import os
import random
import pandas as pd

def find_images(dpath, subdir=''):
    """ return [](filepath, label) """
    images = []
    if os.path.isdir(os.path.join(dpath, subdir)):
        for filename in os.listdir(os.path.join(dpath, subdir)):
            filepath = os.path.join(subdir, filename)
            images.append(filepath)
    return images


def split_images(images, splits):
    'shuffle and split images[] to train[], validation and test[]'
    if type(splits) not in (list, tuple):
        raise TypeError("splits must be list or tuple")
    elif abs(sum(splits) - 1.0) > 0.000001:
        raise ValueError("sum of splits should be 1.0")
    elif len(splits) not in (2, 3):
        raise ValueError("splits should have 2 or 3 elements") 
    random.shuffle(images)
    n = len(images)
    if len(splits) == 2:
        n1, _ = [round(x*n) for x in splits]
        print(n1)
        return images[:n1], images[n1:]
    else:
        n1, n2, _ = [round(x*n) for x in splits]
        return images[:n1], images[n1:n1+n2], images[n1+n2:]

all_classes = next(os.walk('images'))[1]

train = []
val = []
test = []
splits = (0.8, 0.1, 0.1)
for cl in all_classes:
    images = find_images('images', cl)
    tr, v, te = split_images(images, splits)
    train += zip(tr, [cl] * len(tr))
    val += zip(v, [cl] * len(v))
    test += zip(te, [cl] * len(te))

train_df = pd.DataFrame(train, columns=['filename', 'class'])
val_df = pd.DataFrame(val, columns=['filename', 'class'])
test_df = pd.DataFrame(test, columns=['filename', 'class'])


* create data generator for training

In [0]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
seed = 76

classes = ['BEANS', 'CAKE', 'CANDY', 'MILK', 'PASTA']
n_classes = len(classes)
classmap = {c: i for i, c in enumerate(classes)}
img_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [6]:
import os
labels_count = dict()
for img_class in classes:
    labels_count[img_class] = len(os.listdir('images/' + img_class))
total_count = sum(labels_count.values())
class_weights = {cls: total_count / count for cls, count in 
                 enumerate(labels_count.values())}
class_weights

{0: 7.375,
 1: 6.229813664596273,
 2: 2.696236559139785,
 3: 6.191358024691358,
 4: 5.8313953488372094}

In [7]:
train_dataset = train_df[train_df['class'].isin(classes)]
train_filenames = train_dataset['filename'].tolist()
train_labels = train_dataset['class'].tolist()
train_labels = [classmap[l] for l in train_labels]
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=n_classes)

val_dataset = val_df[val_df['class'].isin(classes)]
val_filenames = val_dataset['filename'].tolist()
val_labels = val_dataset['class'].tolist()
val_labels = [classmap[l] for l in val_labels]
val_labels = tf.keras.utils.to_categorical(val_labels, num_classes=n_classes)

test_dataset = test_df[test_df['class'].isin(classes)]
test_filenames = test_dataset['filename'].tolist()
test_labels = test_dataset['class'].tolist()
test_labels = [classmap[l] for l in test_labels]
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=n_classes)

len(train_filenames), len(val_filenames), len(test_filenames)

(804, 100, 99)

In [8]:
# Function to load and preprocess each image
def _parse_fn(filename, label):
    filename = 'images/' + filename
    img = tf.io.read_file(filename)
    img = tf.image.decode_png(img)
    img = (tf.cast(img, tf.float32)/127.5) - 1
    # img = tf.cast(img, tf.float32)/255.0
    img = tf.image.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    return img, label

train_data = tf.data.Dataset.from_tensor_slices(
  (tf.constant(train_filenames), tf.constant(train_labels))
)

val_data = tf.data.Dataset.from_tensor_slices(
  (tf.constant(val_filenames), tf.constant(val_labels))
)

test_data = tf.data.Dataset.from_tensor_slices(
    (tf.constant(test_filenames), tf.constant(test_labels))
)

train_data = (train_data.shuffle(buffer_size=len(train_filenames))
             .map(_parse_fn)
             .batch(BATCH_SIZE)
)

val_data = (val_data.shuffle(buffer_size=len(val_filenames))
            .map(_parse_fn)
            .batch(BATCH_SIZE)
)

test_data = (test_data.map(_parse_fn))

n_train = len(train_filenames)
n_val = len(val_filenames)
n_train, n_val

(804, 100)

 * build train model

In [9]:
import tensorflow as tf
from tensorflow.keras import regularizers

w_l2 = 0.0001

# Pre-trained model with MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_shape,
    include_top=False,
    weights='imagenet'
)

# Freeze the pre-trained model weights
base_model.trainable = False

# Trainable classification head
maxpool_layer = tf.keras.layers.GlobalMaxPooling2D()
fc_layer_1 = tf.keras.layers.Dense(2048, activation='relu')

# Prediction with L2 regularizer
prediction_layer = tf.keras.layers.Dense(
    n_classes, 
    kernel_regularizer=regularizers.l2(w_l2), 
    activation='softmax')

# Layer classification head with feature detector
model = tf.keras.Sequential([
    base_model,
    maxpool_layer,
    fc_layer_1,
    prediction_layer
])

model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_max_pooling2d (Global (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              2623488   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 10245     
Total params: 4,891,717
Trainable params: 2,633,733
Non-trainable params: 2,257,984
_________________________________________________________________


In [10]:
epochs = 3000
learning_rate = 0.0001

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
              loss='categorical_crossentropy',
              metrics=['accuracy']
)

steps_per_epoch = round(n_train/BATCH_SIZE)
validation_steps = round(n_val/BATCH_SIZE)

hist = model.fit(train_data.repeat(), 
                 epochs=epochs,
                 steps_per_epoch=steps_per_epoch,
                 validation_data=val_data.repeat(),
                 validation_steps=validation_steps,
                 class_weight=class_weights)


Train for 25 steps, validate for 3 steps
Epoch 1/3000
25/25 [==============================] - 14s 549ms/step - loss: 3.4707 - accuracy: 0.3900 - val_loss: 1.8377 - val_accuracy: 0.4896
Epoch 2/3000
25/25 [==============================] - 6s 223ms/step - loss: 1.0674 - accuracy: 0.6943 - val_loss: 1.2086 - val_accuracy: 0.5729
Epoch 3/3000
25/25 [==============================] - 5s 203ms/step - loss: 0.5555 - accuracy: 0.8096 - val_loss: 1.7130 - val_accuracy: 0.4688
Epoch 4/3000
25/25 [==============================] - 5s 204ms/step - loss: 0.5308 - accuracy: 0.8264 - val_loss: 1.3487 - val_accuracy: 0.5938
Epoch 5/3000
25/25 [==============================] - 5s 205ms/step - loss: 0.4362 - accuracy: 0.8329 - val_loss: 1.5271 - val_accuracy: 0.5729
Epoch 6/3000
25/25 [==============================] - 5s 207ms/step - loss: 0.4302 - accuracy: 0.8536 - val_loss: 1.4911 - val_accuracy: 0.5625
Epoch 7/3000
25/25 [==============================] - 5s 205ms/step - loss: 0.3228 - accuracy:

In [0]:
import matplotlib.pyplot as plt


# Plot training & validation accuracy values
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.ylim(0.3, 1.1)
plt.show()

# Plot training & validation loss values
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.ylim(0,2)
plt.show()

In [0]:
import numpy as np

x_data = []
y_data = []
for x, y in test_data.repeat(1):
    x_data.append(x)
    y_data.append(y)

x_data = np.vstack(x_data).reshape(-1,224,224,3)
predictions = model.predict_classes(x_data)

for i in range(len(y_data)):
    print(y_data[i], predictions[i], 'x' if list(y_data[i]).index(1) == predictions[i] else '' )


tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 2 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 2 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 1 x
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 2 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 4 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 0 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 1 x
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 4 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 2 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 0 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 4 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 2 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 2 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 4 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 4 
tf.Tensor([0. 1. 0. 0. 0.], shape=(5,), dtype=float32) 4 
tf.Tensor([0. 0. 1. 0. 0.], shape=(5,), dtype=float32) 2 x
tf.Tensor([

In [0]:
model.save('drive/My Drive/Colab Notebooks/grocery-categories-v05-a.h5')
model.save_weights('drive/My Drive/Colab Notebooks/grocery-categories-v05-a.weights.h5')